In [14]:
import warnings
warnings.filterwarnings('ignore')
# We import all our dependencies.
import torch
import sys
import os
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import tifffile as tiff
import numpy as np
import glob

In [16]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

data_dir="/group/jug/Sheida/pancreatic beta cells/download/"
"""
Three train Images -> ['high_c1', 'high_c2', 'high_c3']
'source' is the original image
Four classes -> ['membrane_full_mask', 'nucleus_mask', 'mitochondria_mask', 'granules']
in general I want to first load these 12 ground truth and 3 original images
"""

Three_train_images = ['high_c1', 'high_c2', 'high_c3']
Four_classes = ['membrane_full_mask', 'nucleus_mask', 'granules', 'mitochondria_mask']

# Load source images
train_img_paths = [os.path.join(data_dir, img, f"{img}_source.tif") for img in Three_train_images]
train_images = {img: tiff.imread(path) for img, path in zip(Three_train_images, train_img_paths)}

# Print loaded train images paths
print("Train images loaded from paths:")
for img, path in zip(Three_train_images, train_img_paths):
    print(path)

# Load ground truth images
ground_truth_paths = {img: {cls: os.path.join(data_dir, img, f"{img}_{cls}.tif") for cls in Four_classes} for img in Three_train_images}
ground_truth_images = {img: {cls: tiff.imread(path) for cls, path in cls_paths.items()} for img, cls_paths in ground_truth_paths.items()}

# Print loaded ground truth images paths
print("Ground truth images loaded from paths:")
for img, cls_paths in ground_truth_paths.items():
    for cls, path in cls_paths.items():
        print(path)

# Debug information
print(f"Loaded {len(train_images)} train images.")
print(f"Loaded ground truth images for each class and train image.")

for key, value in train_images.items():
    print(key,value.shape)

for key1 in ground_truth_images.keys():
    for key2, value in ground_truth_images[key1].items():
        print(key1, key2, value.shape)

Train images loaded from paths:
/group/jug/Sheida/pancreatic beta cells/download/high_c1/high_c1_source.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c2/high_c2_source.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c3/high_c3_source.tif
Ground truth images loaded from paths:
/group/jug/Sheida/pancreatic beta cells/download/high_c1/high_c1_membrane_full_mask.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c1/high_c1_nucleus_mask.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c1/high_c1_granules.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c1/high_c1_mitochondria_mask.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c2/high_c2_membrane_full_mask.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c2/high_c2_nucleus_mask.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c2/high_c2_granules.tif
/group/jug/Sheida/pancreatic beta cells/download/high_c2/high_c2_mitochondria_mask.tif
/group/jug/Sheida/pancreatic be

In [17]:
import numpy as np
import tifffile as tiff
import os

# Define data directory
data_dir = "/group/jug/Sheida/pancreatic beta cells/download/"

# Image and class identifiers
Three_train_images = ['high_c1', 'high_c2', 'high_c3']
Four_classes = ['membrane_full_mask', 'nucleus_mask', 'mitochondria_mask', 'granules']

# Function to load images
def load_images(data_dir, image_ids, class_ids):
    ground_truth_paths = {img: {cls: os.path.join(data_dir, img, f"{img}_{cls}.tif") for cls in class_ids} for img in image_ids}
    ground_truth_images = {img: {cls: tiff.imread(path) for cls, path in cls_paths.items()} for img, cls_paths in ground_truth_paths.items()}
    return ground_truth_images

# Load the ground truth images
ground_truth_images = load_images(data_dir, Three_train_images, Four_classes)

# Initialize dictionaries to store the combined labels
combined_labels = {img: np.full_like(ground_truth_images[img]['membrane_full_mask'], -1, dtype=int) for img in Three_train_images}

# Process each image
for img in Three_train_images:
    membrane_mask = ground_truth_images[img]['membrane_full_mask']
    nucleus_mask = ground_truth_images[img]['nucleus_mask']
    mitochondria_mask = ground_truth_images[img]['mitochondria_mask']
    granules_mask = ground_truth_images[img]['granules']

    # Process each slice in the z-stack
    for z in range(membrane_mask.shape[0]):
        mem_slice = membrane_mask[z]
        nuc_slice = nucleus_mask[z]
        mito_slice = mitochondria_mask[z]
        gran_slice = granules_mask[z]

        combined_slice = combined_labels[img][z]

        # Find non-zero elements in membrane_full_mask
        non_zero_indices = np.nonzero(mem_slice)

        for i, j in zip(*non_zero_indices):
            if nuc_slice[i, j] != 0:
                combined_slice[i, j] = 1
            elif gran_slice[i, j] != 0:
                combined_slice[i, j] = 2
            elif mito_slice[i, j] != 0:
                combined_slice[i, j] = 3
            else:
                combined_slice[i, j] = 0

# Debug information
for img in Three_train_images:
    print(f"Processed {img}, combined label shape: {combined_labels[img].shape}")
    unique_labels, counts = np.unique(combined_labels[img], return_counts=True)
    print(f"Unique labels in {img}: {dict(zip(unique_labels, counts))}")


Processed high_c1, combined label shape: (1097, 699, 760)
Unique labels in high_c1: {-1: 336073190, 0: 148661252, 1: 30835445, 2: 50691050, 3: 16509343}
Processed high_c2, combined label shape: (1043, 606, 870)
Unique labels in high_c2: {-1: 346330475, 0: 118060283, 1: 28269712, 2: 37419176, 3: 19810814}
Processed high_c3, combined label shape: (1023, 676, 845)
Unique labels in high_c3: {-1: 365043831, 0: 121570579, 1: 25018115, 2: 61947009, 3: 10778526}


In [19]:
combined_labels.keys()

dict_keys(['high_c1', 'high_c2', 'high_c3'])

In [22]:
for key, value in combined_labels.items():
    tiff.imwrite(data_dir + f"combined_labels_{key}.tif", value.astype(np.int8))